![Spark Image](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Apache_Spark_logo.svg/1200px-Apache_Spark_logo.svg.png)

# Exercise

### This exercise provides an opportunity to apply the concepts and features learned in the previous notebooks.
#### Please replace the placeholders (format '<font color='lightgreen'>###something to replace###</font>') including the '<font color='lightgreen'>#</font>' charaters in the pyspark code according to the description .
#### The solution can be found in the solution notebook - but don't cheat ;)

#### *Create a SparkSession*

Exercise - let's create a *Spark session* - do you remenber which object you have to import?

In [ ]:
# import SparkSession from library
from pyspark.sql import ### library for Spark sessions ###
spark = SparkSession.builder.appName("Exercise").### function to create or replace a Spark session ###

In [ ]:
# display information about the Spark session
### 'command' to display information about the Spark session ###

#### *Load data*

Now we need data to work with. For this we load data that is already provided in the local /data/berlin-data folder.<br><br>
The data was downloaded from the repository: https://github.com/berlinonline/haeufige-vornamen-berlin.<br>
The originator of the data in the original repository folder data/source/ is "Berlin State Office for Citizens and Regulatory Affairs (LABO)".<br>
The originator of the 'cleaned' data in the original repository folder data/cleaned/ is the "Berlin State Office for Citizens' and Regulatory Affairs (LABO) / BerlinOnline Stadtportal GmbH & Co. KG".<br>

All data sets contained in the repository are licensed under CC BY 3.0 DE (Creative Commons Attribution 3.0 Germany License).

##### Data content and data structure

Since 2013, the Berlin city data portal daten.berlin.de has always published lists of the first names of all newborn children and those registered with the registry office at the beginning of the new year. The State Office for Civil and Regulatory Affairs collects the lists from the registry offices in the individual Berlin districts and then publishes them.

There is a folder for each year that contains a CSV file with the frequency of names for each district of Berlin.<br><br>
For the years 2012-2016, the column structure of the CSV files is as follows:
- 'vorname' specifies the first name
- 'anzahl' the total number of children registered with this name
- 'geschlecht' the gender of the child

From 2017 there is an additional column 'position':
- 'position': In the event that a child has been given several first names, position designates the position<br> of the name in the list of names.

*There is nothing to add or exchange in the next code cell!*<br><br> In the next cell, all data in the different subdirectories is read, the information from folder names and file names is added to the datafarm as columns and all data is brought into a uniform schema. Furthermore, all column names and values are converted from German to English and the 'new' dataframe is saved as a file. The file is the basis for the further tasks in this notebook.

In [ ]:
# import lit function from library
from pyspark.sql.functions import lit, translate
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

# create schema for dataframe
csvSchema = StructType([StructField("vorname", StringType(), True),
                             StructField("anzahl", IntegerType(), True),
                             StructField("geschlecht", StringType(), True),
                             StructField("position", IntegerType(), True),
                             StructField("year", IntegerType(), True),
                             StructField("district", StringType(), True) 
                            ])

# create empty dataframe
df = spark.createDataFrame(spark.sparkContext.emptyRDD(), csvSchema)

# 'root' path of berlin data
fileDirectory = 'data/berlin-data/cleaned/'
# loop over all subdirectories (each subdir = one year of data)
for dname in os.listdir(fileDirectory):    
    # loop over all files in the 'year' directory
    fpath = fileDirectory + dname + "/"
    for fname in os.listdir(fpath):
        df_tmp = spark.read.format("csv")\
            .option("header", "true")\
            .option("inferSchema",True)\
            .load(fpath + fname)
        # check if schema contains row 'position'. if not add row with default value 1
        if not ("position" in df_tmp.columns):
                    df_tmp = df_tmp.withColumn("position", lit(1))
        # get the final component of a pathname. This represents the year.
        year = os.path.basename(os.path.dirname(fpath))
        # add the year value column
        df_tmp = df_tmp.withColumn("Year", lit(year))
        # add the disrict value column. The district value is the file name without '.csv' extension.
        df_tmp = df_tmp.withColumn("District", lit(fname[:-4]))
        # add the df_tmp dataframe to the df dataframe 
        df = df.union(df_tmp)

# rename columns from German -> English
df = df.withColumnRenamed("vorname","FirstName") \
    .withColumnRenamed("anzahl","NumberOfChildren")\
    .withColumnRenamed("geschlecht","Gender")\
    .withColumnRenamed("position","Position")\
    .withColumnRenamed("year","Year")\
    .withColumnRenamed("district","District")

# change gender value from w (weiblich) -> f (female)    
df = df.withColumn('Gender', translate('Gender', 'w', 'f'))
# Write DataFrame data to CSV file
df.coalesce(1).write.mode("overwrite").option("header", "true").csv("data/berlin-data/berlin-data")

Exercise - in the next cells you have to 
- read a file in csv format and create a dataframe
- persist the dataframe with the default storage level (MEMORY_AND_DISK)
- print the schema of the dataframe
- display the number of rows in the dataframe
- display the first ten rows of the dataframe

In [ ]:
# read .csv file
df = spark.read.format("csv")\
            .### function add an option to the reader ###("header", "true")\
            .### function add an option to the reader ###("inferSchema",True)\
            .### function to load data ###("data/berlin-data/berlin-data")

In [ ]:
# check if dataframe is already cached - if not cache/persist the dataframe
if not (df.storageLevel.useMemory) :
    df.### function to cache/persist the dataframe ###   

In [ ]:
# print the dataframe schema
df.### function to display the dataframe schema ###

In [ ]:
# display the number of rows in the dataframe
df.### function to display the number of rows in the dataframe ###

In [ ]:
# display the first ten rows of the dataframe
df.### function to display the first ten rows of the dataframe###(10,False)

#### *Analyse data*

We already know how many rows are in the dataframe.<br><br>Exercise - let's find out 
- how many different first names do exist
- how many female and male entries exist
- how many entries are made for each year
- what is the most popular female first name
- what is the most popular male first name
- the maximum number of first name in the register (maximum position)
- which district has the most entries
- How many children have the same name (NumberOfChildren) in average in the year 2021?

In [ ]:
# how many different first names do exist?

# import ??? function from library
from pyspark.sql.functions import ### function to count distinct values ###
df.select(### function to count distinct values ###("FirstName")).show()

# alternative implementation (return type is integer)
df.select("FirstName").distinct().count()

In [ ]:
# how many different female and male entries exist?
df.### function to group the data ###("Gender").count().show()

In [ ]:
# how many entries are made for each year?
df.sort("Year").### function to group the data ###(df.Year).count().show()

In [ ]:
# what is the most popular female first name?
df.sort(df.NumberOfChildren.desc()).### function to apply a filter ###(df.Gender == "f").show(5, False)

In [ ]:
# what is the most popular female first name?
df.sort(df.NumberOfChildren.desc()).### function to apply a filter ###(df.Gender == "m").show(5, False)

In [ ]:
# what is the maximum number of first names somebody has (maximum position)?

# import ??? function from library
from pyspark.sql.functions import ### function to get maximum value ###
df.select(### function to get maximum value ###("Position")).show()

In [ ]:
# which district has the most entries?

# import functions from library
from pyspark.sql.functions import ### function to get a column from name ###
df.groupBy("District").count().sort(### function to get a column from name ###("count").desc()).show(truncate = False)

In [ ]:
# How many children have the same name (NumberOfChildren) in average in the year 2021?

# import functions from library
from pyspark.sql.functions import ### function to get average value ###
df.filter(df.Year == '2021').### function to compute aggregates ###(### function to get average value ###("NumberOfChildren")).show()

#### *Analyse data using 'sql' and 'join' commands*

Beside the functions used in above chapter we know want to use 'join' commands.<br><br>
Exercise - let's find out 
- how many first names exist in dictrict mitte but not in district pankow
- how many female and male entries exist
- how many entries are made for each year
- what is the most popular female first name
- what is the most popular male first name
- the maximum number of first name in the register (maximum position)
- which district has the most entries

In [ ]:
# to have 'easier' acces to the different data of the districts 
# we can create new dataframe for the relevant districts

df_mitte = df.filter(df.District == 'mitte')
df_pankow = df.filter(df.District == 'pankow')
df_lichtenberg = df.filter(df.District == 'lichtenberg')

In [ ]:
# how many first names exist in dictrict mitte but not in district pankow? 
# use dataframe 'join' command

df_mitte.join(df_pankow, (df_mitte.FirstName == df_pankow.FirstName), ### add join type in quotes (as string) ###).count()

In [ ]:
# how many first names exist in dictrict mitte but not in district pankow? 
# use 'sql' command - hint: don't forget to create local temporary views

df_mitte.### function to create or replace a temporary view ###("mitte")
df_pankow.### function to create or replace a temporary view ###("pankow")

joinDF = spark.### function to use SQL statements ###("SELECT count(*) FROM mitte m \
                    LEFT OUTER JOIN pankow p \
                    ON m.FirstName = p.FirstName \
                    WHERE p.FirstName IS NULL ") \
  .show(truncate=False)